# Model Training

## Prerequisites

To run this notebook the `data_path` variable has to point to the location of a valid dataset. If you don't have any data an example dataset can be obtained by characterizing [PTM](http://ptm.asu.edu/) transistor models as shown in [pyrdict](https://github.com/AugustUnderground/pyrdict).

Additionally, for training [precept](https://github.com/electronics-and-drives/precept) has to be installed manually.

## Setup

In [1]:
%matplotlib widget

In [2]:
import os
import torch
import h5py as h5
import numpy as np
import torch as pt
import pandas as pd
import joblib as jl
from datetime import datetime as dt
from functools import partial
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, minmax_scale

In [3]:
from pytorch_lightning import Trainer
from precept import PreceptModule, PreceptDataFrameModule

In [4]:
num_gpus = pt.cuda.device_count() if pt.cuda.is_available() else None

## Data

**Note**: This part has to be adjusted according to the location and shape of custom datasets.

- Adjust `data_path` accordingly, so it points to a your $\frac{g_{\mathrm{m}}}{I_{\mathrm{d}}}$ look up table.
- Depending on the shape of the dataset and file format it may need to be read differently.

If the data is generated with [pyrdict](https://github.com/AugustUnderground/pyrdict) the resulting `h5` is structured such that each parameter is a group.

In [5]:
device_type = "nmos"
device_tech = "90nm"
device_name = "ptm"
data_path   = f"../data/{device_tech}_bulk_{device_type}.h5"

In [6]:
with h5.File(data_path, "r") as data_base:
    column_names = list(data_base.keys())
    data_matrix  = np.transpose(np.array([data_base[c] for c in column_names]))
    data_frame   = pd.DataFrame(data_matrix, columns=column_names).dropna()

### Preprocesing and Filtering

Only the data necessary for preprocessing and the function mappings is kept, everything else can be filtered out.

In [7]:
params   = [ "Vgs", "Vds", "Vbs", "L", "W"
           , "gmid", "fug", "vdsat", "a0"
           , "jd", "id", "gm", "gds" ]
data_raw = data_frame[params]

Since in the vast majority of cases a device is desired to be in saturation all other cases are filtered out.

In [8]:
data_filtered = data_raw[( (round(data_raw.Vgs, 2) == round(data_raw.Vds, 2))
                         & (round(data_raw.Vbs, 2) == 0.0)
                         & (data_raw.W == np.random.choice(data_raw.W.unique())))].sample(frac=1)
data_filtered.describe()

,Vgs,Vds,Vbs,L,W,gmid,fug,vdsat,a0,jd,id,gm,gds
count,1200.000000,1200.000000,1.200000e+03,1.200000e+03,1.200000e+03,1200.000000,1.200000e+03,1200.000000,1200.000000,1200.000000,1.200000e+03,1.200000e+03,1.200000e+03
mean,0.604999,0.605000,8.661447e-10,5.075000e-06,1.744444e-05,11.808823,6.372850e+09,0.252605,308.240208,30.788507,5.370884e-04,1.518598e-03,3.565746e-05
std,0.346542,0.346543,3.186241e-09,3.144863e-06,3.389544e-21,9.293700,2.246736e+10,0.202460,186.648489,97.920047,1.708161e-03,3.792286e-03,1.348778e-04
min,0.010000,0.010000,2.476256e-14,1.500000e-07,1.744444e-05,1.401892,3.244685e+02,0.041947,0.312317,0.000003,5.903050e-11,1.578751e-09,9.236715e-10
25%,0.307500,0.307500,6.004121e-13,2.338889e-06,1.744444e-05,3.105788,7.796671e+06,0.049042,164.455001,0.049738,8.676517e-07,1.856484e-05,5.106374e-08
50%,0.605000,0.605000,1.357500e-12,5.075000e-06,1.744444e-05,7.858912,3.909244e+07,0.204440,317.302911,3.724268,6.496778e-05,4.021849e-04,9.551545e-07
75%,0.902499,0.902500,4.712170e-11,7.811111e-06,1.744444e-05,21.654459,1.896145e+08,0.424195,447.171523,15.570802,2.716240e-04,8.739644e-04,3.158873e-06
max,1.199995,1.200000,3.211726e-08,1.000000e-05,1.744444e-05,26.747242,9.755854e+10,0.671235,675.272900,717.984270,1.252484e-02,1.757258e-02,7.986603e-04


### Mappings

For each device ($\delta \in [\mathrm{"NMOS"}, \mathrm{"PMOS"}]$) and 
technology ($\tau = 90\,\mathrm{nm}$ in this case) the mapping $\gamma$ is trained

$$\gamma_{\mathrm{\delta,\tau}} = 
    \begin{bmatrix} 
        \frac{g_{\mathrm{m}}}{I_{\mathrm{d}}} \\ f_{\mathrm{ug}} \\ A_{0}
    \end{bmatrix} 
    \mapsto
    \begin{bmatrix}
        J_{\mathrm{d}} \\ L
    \end{bmatrix},$$

where $A_{0} = \frac{g_{\mathrm{m}}}{g_{\mathrm{ds}}}$ and $J_{\mathrm{d}} = \frac{I_{\mathrm{d}}}{W}$.

The model converges better when $f_{\mathrm{ug}}$ and $J_{\mathrm{d}}$ are trained on the $\log_{10}$ instead.

In [14]:
transformed_x = np.vstack([ data_filtered.gmid.values
                          , np.log10(data_filtered.fug.values)])
transformed_y = np.vstack([ np.log10(data_filtered.jd.values)
                          , data_filtered.L.values
                          , data_filtered.a0.values ])

In [16]:
transformed_x.shape

(2, 1200)

In [17]:
data_filtered[["gmid", "fug"]].values.shape

(1200, 2)

Additionally, the data is scaled such that each parameter is $\in [0;1]$.

In [18]:
scale_x     = MinMaxScaler()
scale_y     = MinMaxScaler()

scaled_x    = scale_x.fit_transform(transformed_x.T).T
scaled_y    = scale_y.fit_transform(transformed_y.T).T

## Training 

The batch size is deliberatly chosen to be small, based on experience with other datasets but may be increased at will. Same goes for the number of epochs.

In [19]:
batch_size = 2000
test_split = 0.2
num_epochs = 42
params_x = ["gmid", "fug"]
params_y = ["jd", "L", "a0"]

In [20]:
data = pd.DataFrame( np.vstack((scaled_x, scaled_y)).T
                   , columns = (params_x + params_y) )

A folder for storing the model will be dynamically created in the root of this repositry each time the notebook is run.
All trained models will be located under `models/custom/` and assigned with a timestamp.

In [22]:
time_stamp = dt.now().strftime("%Y-%m%d-%H%M%S")
model_dir  = f"../models/custom/{device_tech}-{device_name}-{time_stamp}/"
os.makedirs(model_dir, exist_ok = True)
model_prefix = f"{model_dir}/v-{device_type}-{device_tech}"

The data module handles loading and preparing the dataset for training.

In [23]:
data = PreceptDataFrameModule( data                    # unecessary columns are filtered out internally
                             , params_x, params_y      # input and output parameters
                             , [], [], [], []          # No transformations
                             , batch_size = batch_size
                             , test_split = test_split
                             , scale      = False      # No need to scale data, we did that before
                             , )

The lightning module prepares the Neural Network.

In [25]:
module = PreceptModule( len(params_x), len(params_y)
                      , model_path = model_dir
                      , )

The Trainier handles training, logging and periodic checkpoint saving of the model.

In [26]:
trainer     = Trainer( gpus                = num_gpus
                     , max_epochs          = num_epochs
                     , precision           = 64
                     , checkpoint_callback = True
                     , default_root_dir    = model_dir
                     , )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Use the trainer to fit the module on the given data.

In [27]:
trainer.fit(module, data)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 1.4 M 
------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
11.038    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Save the best model and scalers for later use in circuit sizing.

In [29]:
trainer.save_checkpoint(f"{model_prefix}.cktp")
jl.dump(scale_x, f"{model_prefix}.X")
jl.dump(scale_y, f"{model_prefix}.Y")

['../models/custom/90nm-ptm-2021-0625-192110//v-nmos-90nm.Y']

## Evaluation

The model is evaluated by plotting a trace vs. the ground truth of the data base.